In [4]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

In [3]:
z_py = pd.read_csv('example_results/slide_py/0.1_0.5_out/z_matrix.csv', index_col=0)
z_py.head(3)

,Z0,Z1,Z2,Z3,Z4,Z5,Z6,Z7,Z8,Z9,...,Z123,Z124,Z125,Z126,Z127,Z128,Z129,Z130,Z131,Z132
8W_all_all_CTTTGCGAGCCGATTT-1,1.848815,0.405666,0.026771,-0.877819,0.219456,-0.478235,0.010317,-0.876654,-0.111018,-0.335007,...,-0.095349,0.487235,-0.753535,-0.124726,0.140843,-0.343909,-0.051867,-0.254454,-0.214318,-0.521624
14W_all_all_CCACCTAGTGATGATA-1,-3.168566,-2.627408,-0.823464,0.957615,-0.785026,2.102733,-0.115395,-2.048519,-0.020054,-0.472545,...,-0.096932,-1.690590,2.829873,-0.063814,-0.117057,-0.092239,-0.331353,-1.054934,0.053426,1.253213
10W_all_all_AACTCAGAGACAAGCC-1,-0.830686,-0.334963,0.115307,-0.056327,-0.810581,-0.271599,-0.045089,0.823157,-0.185789,0.455174,...,-0.167658,-0.383332,0.164344,-0.214946,0.821987,0.315426,-0.087000,-0.268614,0.583113,-0.643397


In [6]:
z_R = pd.read_csv('example_results/slide/0.1_0.5_out/z_matrix.csv', index_col=0)
z_R.head(3)


,Z1,Z2,Z3,Z4,Z5,Z6,Z7,Z8,Z9,Z10,...,Z124,Z125,Z126,Z127,Z128,Z129,Z130,Z131,Z132,Z133
8W_all_all_CTTTGCGAGCCGATTT-1,1.844739,0.404772,0.026712,-0.875884,0.218972,-0.477180,0.010294,-0.874721,-0.110773,-0.334269,...,-0.095139,0.486161,-0.751873,-0.124451,0.140533,-0.343151,-0.051753,-0.253893,-0.213846,-0.520473
14W_all_all_CCACCTAGTGATGATA-1,-3.161579,-2.621614,-0.821649,0.955503,-0.783295,2.098097,-0.115140,-2.044002,-0.020010,-0.471503,...,-0.096718,-1.686862,2.823633,-0.063673,-0.116799,-0.092036,-0.330622,-1.052608,0.053308,1.250450
10W_all_all_AACTCAGAGACAAGCC-1,-0.828855,-0.334224,0.115052,-0.056203,-0.808793,-0.271000,-0.044989,0.821342,-0.185379,0.454170,...,-0.167289,-0.382487,0.163982,-0.214472,0.820175,0.314730,-0.086809,-0.268022,0.581827,-0.641979


In [7]:
np.max(abs(z_R.values - z_py.values))

np.float64(0.034612234594662894)